<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20251126%5D%20RF-LSTM%20ROC%20Operating%20Points%3A%20RF-LSTM%20Performance%20Across%20Benchmark%20Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import rcParams

# Set publication-quality parameters
plt.style.use('seaborn-v0_8-paper')
rcParams.update({
    #'font.family': 'serif',
    #'font.serif': ['Times New Roman'],
    'font.size': 10,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'legend.fontsize': 9,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'figure.figsize': (12, 4),
    'mathtext.fontset': 'stix',
})

def create_roc_figures():
    # Load your data
    wsn_bfsf = pd.read_csv('roc_rf_lstm_wsnbfsf.csv')
    wsn_ds = pd.read_csv('roc_rf_lstm_wsnds.csv')
    unsw = pd.read_csv('roc_rf_lstm_unsw.csv')

    # Clean column names and convert to numeric for wsn_bfsf
    wsn_bfsf.columns = wsn_bfsf.columns.str.strip()
    if 'Unnamed: 1' in wsn_bfsf.columns:
        wsn_bfsf['Unnamed: 1'] = pd.to_numeric(wsn_bfsf['Unnamed: 1'], errors='coerce')
    if 'Unnamed: 2' in wsn_bfsf.columns:
        wsn_bfsf['Unnamed: 2'] = pd.to_numeric(wsn_bfsf['Unnamed: 2'], errors='coerce')
    wsn_bfsf.dropna(subset=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

    # Clean column names and convert to numeric for wsn_ds
    wsn_ds.columns = wsn_ds.columns.str.strip()
    if 'Unnamed: 1' in wsn_ds.columns:
        wsn_ds['Unnamed: 1'] = pd.to_numeric(wsn_ds['Unnamed: 1'], errors='coerce')
    if 'Unnamed: 2' in wsn_ds.columns:
        wsn_ds['Unnamed: 2'] = pd.to_numeric(wsn_ds['Unnamed: 2'], errors='coerce')
    wsn_ds.dropna(subset=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

    # Clean column names and convert to numeric for unsw
    unsw.columns = unsw.columns.str.strip()
    if 'Unnamed: 1' in unsw.columns:
        unsw['Unnamed: 1'] = pd.to_numeric(unsw['Unnamed: 1'], errors='coerce')
    if 'Unnamed: 2' in unsw.columns:
        unsw['Unnamed: 2'] = pd.to_numeric(unsw['Unnamed: 2'], errors='coerce')
    unsw.dropna(subset=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

    print("WSN-BFSF columns:", wsn_bfsf.columns.tolist())
    print("WSN-DS columns:", wsn_ds.columns.tolist())
    print("UNSW columns:", unsw.columns.tolist())

    # Create figure with 3 subplots
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

    # Define class labels for each dataset
    wsn_bfsf_labels = ['Normal', 'Blackhole', 'Forwarding', 'Flooding']
    wsn_ds_labels = ['Normal', 'Blackhole', 'Flooding', 'Grayhole', 'TDMA']
    unsw_labels = ['Normal', 'Analysis', 'Backdoor', 'DoS', 'Exploits',
                   'Fuzzers', 'Generic', 'Reconnaissance', 'Shellcode', 'Worms']

    # Define markers for grayscale differentiation
    markers = ['o', 's', '^', 'D', 'v', '>', '<', 'p', '*', 'h']

    # Plot WSN-BFSF (zoomed for near-perfect performance)
    ax1 = axes[0]
    for i, (_, row) in enumerate(wsn_bfsf.iterrows()):
        if i < len(wsn_bfsf_labels):
            ax1.scatter(row['Unnamed: 1'], row['Unnamed: 2'],
                       marker=markers[i], s=80,
                       facecolor='white', edgecolor='black',
                       linewidth=1.5, label=wsn_bfsf_labels[i])

    # Add diagonal reference line
    ax1.plot([0, 0.001], [0, 0.001], 'k--', alpha=0.7, linewidth=1, label='Random Classifier')

    ax1.set_xlim(0, 0.0005)
    ax1.set_ylim(0.999, 1.0001)
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('WSN-BFSF Dataset\n(Near-Perfect Detection)', fontweight='bold')
    ax1.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    ax1.legend(loc='lower right', framealpha=0.9)

    # Plot WSN-DS (zoomed for excellent performance)
    ax2 = axes[1]
    for i, (_, row) in enumerate(wsn_ds.iterrows()):
        if i < len(wsn_ds_labels):
            ax2.scatter(row['Unnamed: 1'], row['Unnamed: 2'],
                       marker=markers[i], s=80,
                       facecolor='white', edgecolor='black',
                       linewidth=1.5, label=wsn_ds_labels[i])

    ax2.plot([0, 0.001], [0, 0.001], 'k--', alpha=0.7, linewidth=1, label='Random Classifier')

    ax2.set_xlim(0, 0.0003)
    ax2.set_ylim(0.997, 1.0001)
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.set_title('WSN-DS Dataset\n(Excellent Detection)', fontweight='bold')
    ax2.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    ax2.legend(loc='lower right', framealpha=0.9)

    # Plot UNSW-NB15 (full scale showing realistic variation)
    ax3 = axes[2]
    for i, (_, row) in enumerate(unsw.iterrows()):
        if i < len(unsw_labels):
            ax3.scatter(row['Unnamed: 1'], row['Unnamed: 2'],
                       marker=markers[i], s=80,
                       facecolor='white', edgecolor='black',
                       linewidth=1.5, label=f"{unsw_labels[i]}\n(AUC: {row['Unnamed: 2']:.3f})")

    ax3.plot([0, 0.05], [0, 0.05], 'k--', alpha=0.7, linewidth=1, label='Random Classifier')

    ax3.set_xlim(0, 0.04)
    ax3.set_ylim(0.5, 1.02)
    ax3.set_xlabel('False Positive Rate')
    ax3.set_ylabel('True Positive Rate')
    ax3.set_title('UNSW-NB15 Dataset\n(Cross-Environment Generalization)', fontweight='bold')
    ax3.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    ax3.legend(loc='lower right', framealpha=0.9, fontsize=8)

    # Adjust layout and add overall title
    plt.suptitle('ROC Operating Points: RF-LSTM Performance Across Benchmark Datasets',
                 fontsize=14, fontweight='bold', y=0.98)
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Save in multiple formats
    plt.savefig('roc_comparison_mobisys.png', bbox_inches='tight', dpi=300)
    plt.savefig('roc_comparison_mobisys.pdf', bbox_inches='tight')
    plt.savefig('roc_comparison_mobisys.eps', bbox_inches='tight', format='eps')

    plt.show()

    # Print performance summary
    print("Performance Summary:")
    print(f"WSN-BFSF: TPR range [{wsn_bfsf['Unnamed: 2'].min():.4f}, {wsn_bfsf['Unnamed: 2'].max():.4f}], "
          f"FPR range [{wsn_bfsf['Unnamed: 1'].min():.6f}, {wsn_bfsf['Unnamed: 1'].max():.6f}]")
    print(f"WSN-DS: TPR range [{wsn_ds['Unnamed: 2'].min():.4f}, {wsn_ds['Unnamed: 2'].max():.4f}], "
          f"FPR range [{wsn_ds['Unnamed: 1'].min():.6f}, {wsn_ds['Unnamed: 1'].max():.6f}]")
    print(f"UNSW-NB15: TPR range [{unsw['Unnamed: 2'].min():.4f}, {unsw['Unnamed: 2'].max():.4f}], "
          f"FPR range [{unsw['Unnamed: 1'].min():.6f}, {unsw['Unnamed: 1'].max():.6f}]")

# Alternative version with micro-average AUC curves
def create_roc_curves_comparison():
    """Create a complementary figure showing the overall performance comparison"""

    # Load data
    wsn_bfsf = pd.read_csv('roc_rf_lstm_wsnbfsf.csv')
    wsn_ds = pd.read_csv('roc_rf_lstm_wsnds.csv')
    unsw = pd.read_csv('roc_rf_lstm_unsw.csv')

    # Clean column names and convert to numeric for wsn_bfsf
    wsn_bfsf.columns = wsn_bfsf.columns.str.strip()
    if 'Unnamed: 1' in wsn_bfsf.columns:
        wsn_bfsf['Unnamed: 1'] = pd.to_numeric(wsn_bfsf['Unnamed: 1'], errors='coerce')
    if 'Unnamed: 2' in wsn_bfsf.columns:
        wsn_bfsf['Unnamed: 2'] = pd.to_numeric(wsn_bfsf['Unnamed: 2'], errors='coerce')
    wsn_bfsf.dropna(subset=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

    # Clean column names and convert to numeric for wsn_ds
    wsn_ds.columns = wsn_ds.columns.str.strip()
    if 'Unnamed: 1' in wsn_ds.columns:
        wsn_ds['Unnamed: 1'] = pd.to_numeric(wsn_ds['Unnamed: 1'], errors='coerce')
    if 'Unnamed: 2' in wsn_ds.columns:
        wsn_ds['Unnamed: 2'] = pd.to_numeric(wsn_ds['Unnamed: 2'], errors='coerce')
    wsn_ds.dropna(subset=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

    # Clean column names and convert to numeric for unsw
    unsw.columns = unsw.columns.str.strip()
    if 'Unnamed: 1' in unsw.columns:
        unsw['Unnamed: 1'] = pd.to_numeric(unsw['Unnamed: 1'], errors='coerce')
    if 'Unnamed: 2' in unsw.columns:
        unsw['Unnamed: 2'] = pd.to_numeric(unsw['Unnamed: 2'], errors='coerce')
    unsw.dropna(subset=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

    fig, ax = plt.subplots(1, 1, figsize=(6, 5))

    # Calculate micro-average AUC for each dataset
    datasets = [
        ('WSN-BFSF', wsn_bfsf, 's', 0.9998),
        ('WSN-DS', wsn_ds, '^', 0.9996),
        ('UNSW-NB15', unsw, 'o', 0.9371)
    ]

    for name, df, marker, auc in datasets:
        # For demonstration, create a simple ROC curve from the operating points
        fpr = df['Unnamed: 1'].values
        tpr = df['Unnamed: 2'].values

        # Sort by FPR for better visualization
        sort_idx = np.argsort(fpr)
        fpr_sorted = fpr[sort_idx]
        tpr_sorted = tpr[sort_idx]

        ax.plot(fpr_sorted, tpr_sorted, marker=marker, linewidth=2,
                markersize=8, label=f'{name} (micro-AUC: {auc:.4f})')

    ax.plot([0, 1], [0, 1], 'k--', alpha=0.6, linewidth=1, label='Random Classifier')
    ax.set_xlim(0, 0.05)
    ax.set_ylim(0.5, 1.02)
    ax.set_xlabel('False Positive Rate', fontweight='bold')
    ax.set_ylabel('True Positive Rate', fontweight='bold')
    ax.set_title('Overall ROC Performance: RF-LSTM Cross-Dataset Comparison',
                 fontweight='bold', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='lower right', framealpha=0.9)

    plt.tight_layout()
    plt.savefig('roc_overall_comparison.png', bbox_inches='tight', dpi=300)
    plt.savefig('roc_overall_comparison.pdf', bbox_inches='tight')
    plt.show()

# Execute the main function
if __name__ == "__main__":
    create_roc_figures()
    create_roc_curves_comparison()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import rcParams

# Set publication-quality parameters
plt.style.use('seaborn-v0_8-paper')
rcParams.update({
    #'font.family': 'serif',
    #'font.serif': ['Times New Roman'],
    'font.size': 15,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'legend.fontsize': 9,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'figure.figsize': (12, 4),
    'mathtext.fontset': 'stix',
})

def create_roc_figures():
    # Load your data
    wsn_bfsf = pd.read_csv('roc_rf_lstm_wsnbfsf.csv')
    wsn_ds = pd.read_csv('roc_rf_lstm_wsnds.csv')
    unsw = pd.read_csv('roc_rf_lstm_unsw.csv')

    # Clean column names and convert to numeric for wsn_bfsf
    wsn_bfsf.columns = wsn_bfsf.columns.str.strip()
    if 'Unnamed: 1' in wsn_bfsf.columns:
        wsn_bfsf['Unnamed: 1'] = pd.to_numeric(wsn_bfsf['Unnamed: 1'], errors='coerce')
    if 'Unnamed: 2' in wsn_bfsf.columns:
        wsn_bfsf['Unnamed: 2'] = pd.to_numeric(wsn_bfsf['Unnamed: 2'], errors='coerce')
    wsn_bfsf.dropna(subset=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

    # Clean column names and convert to numeric for wsn_ds
    wsn_ds.columns = wsn_ds.columns.str.strip()
    if 'Unnamed: 1' in wsn_ds.columns:
        wsn_ds['Unnamed: 1'] = pd.to_numeric(wsn_ds['Unnamed: 1'], errors='coerce')
    if 'Unnamed: 2' in wsn_ds.columns:
        wsn_ds['Unnamed: 2'] = pd.to_numeric(wsn_ds['Unnamed: 2'], errors='coerce')
    wsn_ds.dropna(subset=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

    # Clean column names and convert to numeric for unsw
    unsw.columns = unsw.columns.str.strip()
    if 'Unnamed: 1' in unsw.columns:
        unsw['Unnamed: 1'] = pd.to_numeric(unsw['Unnamed: 1'], errors='coerce')
    if 'Unnamed: 2' in unsw.columns:
        unsw['Unnamed: 2'] = pd.to_numeric(unsw['Unnamed: 2'], errors='coerce')
    unsw.dropna(subset=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

    print("WSN-BFSF columns:", wsn_bfsf.columns.tolist())
    print("WSN-DS columns:", wsn_ds.columns.tolist())
    print("UNSW columns:", unsw.columns.tolist())

    # Create figure with 3 subplots
    #fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Define class labels for each dataset
    wsn_bfsf_labels = ['Normal', 'Blackhole', 'Forwarding', 'Flooding']
    wsn_ds_labels = ['Normal', 'Blackhole', 'Flooding', 'Grayhole', 'TDMA']
    unsw_labels = ['Normal', 'Analysis', 'Backdoor', 'DoS', 'Exploits',
                   'Fuzzers', 'Generic', 'Reconnaissance', 'Shellcode', 'Worms']

    # Define markers for grayscale differentiation
    markers = ['o', 's', '^', 'D', 'v', '>', '<', 'p', '*', 'h']

    # Plot WSN-BFSF (zoomed for near-perfect performance)
    ax1 = axes[0]
    for i, (_, row) in enumerate(wsn_bfsf.iterrows()):
        if i < len(wsn_bfsf_labels):
            ax1.scatter(row['Unnamed: 1'], row['Unnamed: 2'],
                       marker=markers[i], s=80,
                       facecolor='white', edgecolor='black',
                       #linewidth=1.5, label=wsn_bfsf_labels[i])
                       linewidth=1.5, label=f"{unsw_labels[i]}\n(AUC: {row['Unnamed: 2']:.8f})")
    # Add diagonal reference line
    ax1.plot([0, 0.001], [0, 0.001], 'k--', alpha=0.7, linewidth=1, label='Random Classifier')

    ax1.set_xlim(0, 0.0005)
    ax1.set_ylim(0.999, 1.0001)
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('WSN-BFSF Dataset\n(Near-Perfect Detection)', fontweight='bold')
    ax1.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    ax1.legend(loc='lower left', framealpha=0.9, fontsize=5)

    # Plot WSN-DS (zoomed for excellent performance)
    ax2 = axes[1]
    for i, (_, row) in enumerate(wsn_ds.iterrows()):
        if i < len(wsn_ds_labels):
            ax2.scatter(row['Unnamed: 1'], row['Unnamed: 2'],
                       marker=markers[i], s=80,
                       facecolor='white', edgecolor='black',
                       #linewidth=1.5, label=wsn_ds_labels[i])
                       linewidth=1.5, label=f"{unsw_labels[i]}\n(AUC: {row['Unnamed: 2']:.8f})")

    ax2.plot([0, 0.001], [0, 0.001], 'k--', alpha=0.7, linewidth=1, label='Random Classifier')

    ax2.set_xlim(0, 0.0003)
    ax2.set_ylim(0.997, 1.0001)
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.set_title('WSN-DS Dataset\n(Excellent Detection)', fontweight='bold')
    ax2.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    ax2.legend(loc='lower left', framealpha=0.9, fontsize=5)

    # Plot UNSW-NB15 (full scale showing realistic variation)
    ax3 = axes[2]
    for i, (_, row) in enumerate(unsw.iterrows()):
        if i < len(unsw_labels):
            ax3.scatter(row['Unnamed: 1'], row['Unnamed: 2'],
                       marker=markers[i], s=80,
                       facecolor='white', edgecolor='black',
                       linewidth=1.5, label=f"{unsw_labels[i]}\n(AUC: {row['Unnamed: 2']:.8f})")

    ax3.plot([0, 0.05], [0, 0.05], 'k--', alpha=0.7, linewidth=1, label='Random Classifier')

    ax3.set_xlim(0, 0.04)
    ax3.set_ylim(0.5, 1.02)
    ax3.set_xlabel('False Positive Rate')
    ax3.set_ylabel('True Positive Rate')
    ax3.set_title('UNSW-NB15 Dataset\n(Cross-Environment Generalization)', fontweight='bold')
    ax3.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    ax3.legend(loc='lower left', framealpha=0.9, fontsize=5)

    # Adjust layout and add overall title
    plt.suptitle('ROC Operating Points: RF-LSTM Performance Across Benchmark Datasets',
                 fontsize=20, fontweight='bold', y=0.95)
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Save in multiple formats
    plt.savefig('roc_comparison_mobisys.png', bbox_inches='tight', dpi=300)
    plt.savefig('roc_comparison_mobisys.pdf', bbox_inches='tight')
    plt.savefig('roc_comparison_mobisys.eps', bbox_inches='tight', format='eps')

    plt.show()

    # Print performance summary
    print("Performance Summary:")
    print(f"WSN-BFSF: TPR range [{wsn_bfsf['Unnamed: 2'].min():.4f}, {wsn_bfsf['Unnamed: 2'].max():.4f}], "
          f"FPR range [{wsn_bfsf['Unnamed: 1'].min():.6f}, {wsn_bfsf['Unnamed: 1'].max():.6f}]")
    print(f"WSN-DS: TPR range [{wsn_ds['Unnamed: 2'].min():.4f}, {wsn_ds['Unnamed: 2'].max():.4f}], "
          f"FPR range [{wsn_ds['Unnamed: 1'].min():.6f}, {wsn_ds['Unnamed: 1'].max():.6f}]")
    print(f"UNSW-NB15: TPR range [{unsw['Unnamed: 2'].min():.4f}, {unsw['Unnamed: 2'].max():.4f}], "
          f"FPR range [{unsw['Unnamed: 1'].min():.6f}, {unsw['Unnamed: 1'].max():.6f}]")

# Execute the main function
if __name__ == "__main__":
    create_roc_figures()

In [ ]:
# Alternative version with micro-average AUC curves
def create_roc_curves_comparison():
    """Create a complementary figure showing the overall performance comparison"""

    # Load data
    wsn_bfsf = pd.read_csv('roc_rf_lstm_wsnbfsf.csv')
    wsn_ds = pd.read_csv('roc_rf_lstm_wsnds.csv')
    unsw = pd.read_csv('roc_rf_lstm_unsw.csv')

    # Clean column names and convert to numeric for wsn_bfsf
    wsn_bfsf.columns = wsn_bfsf.columns.str.strip()
    if 'Unnamed: 1' in wsn_bfsf.columns:
        wsn_bfsf['Unnamed: 1'] = pd.to_numeric(wsn_bfsf['Unnamed: 1'], errors='coerce')
    if 'Unnamed: 2' in wsn_bfsf.columns:
        wsn_bfsf['Unnamed: 2'] = pd.to_numeric(wsn_bfsf['Unnamed: 2'], errors='coerce')
    wsn_bfsf.dropna(subset=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

    # Clean column names and convert to numeric for wsn_ds
    wsn_ds.columns = wsn_ds.columns.str.strip()
    if 'Unnamed: 1' in wsn_ds.columns:
        wsn_ds['Unnamed: 1'] = pd.to_numeric(wsn_ds['Unnamed: 1'], errors='coerce')
    if 'Unnamed: 2' in wsn_ds.columns:
        wsn_ds['Unnamed: 2'] = pd.to_numeric(wsn_ds['Unnamed: 2'], errors='coerce')
    wsn_ds.dropna(subset=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

    # Clean column names and convert to numeric for unsw
    unsw.columns = unsw.columns.str.strip()
    if 'Unnamed: 1' in unsw.columns:
        unsw['Unnamed: 1'] = pd.to_numeric(unsw['Unnamed: 1'], errors='coerce')
    if 'Unnamed: 2' in unsw.columns:
        unsw['Unnamed: 2'] = pd.to_numeric(unsw['Unnamed: 2'], errors='coerce')
    unsw.dropna(subset=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

    fig, ax = plt.subplots(1, 1, figsize=(6, 5))

    # Calculate micro-average AUC for each dataset
    datasets = [
        ('WSN-BFSF', wsn_bfsf, 's', 0.9998),
        ('WSN-DS', wsn_ds, '^', 0.9996),
        ('UNSW-NB15', unsw, 'o', 0.9371)
    ]

    for name, df, marker, auc in datasets:
        # For demonstration, create a simple ROC curve from the operating points
        fpr = df['Unnamed: 1'].values
        tpr = df['Unnamed: 2'].values

        # Sort by FPR for better visualization
        sort_idx = np.argsort(fpr)
        fpr_sorted = fpr[sort_idx]
        tpr_sorted = tpr[sort_idx]

        ax.plot(fpr_sorted, tpr_sorted, marker=marker, linewidth=2,
                markersize=8, label=f'{name} (micro-AUC: {auc:.8f})')

    ax.plot([0, 1], [0, 1], 'k--', alpha=0.6, linewidth=1, label='Random Classifier')
    ax.set_xlim(0, 0.05)
    ax.set_ylim(0.5, 1.02)
    ax.set_xlabel('False Positive Rate', fontweight='bold')
    ax.set_ylabel('True Positive Rate', fontweight='bold')
    ax.set_title('Overall ROC Performance: RF-LSTM Cross-Dataset Comparison',
                 fontweight='bold', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='lower right', framealpha=0.9)

    plt.tight_layout()
    plt.savefig('roc_overall_comparison.png', bbox_inches='tight', dpi=300)
    plt.savefig('roc_overall_comparison.pdf', bbox_inches='tight')
    plt.show()

create_roc_curves_comparison()